In [6]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### y8.com -- get categories

In [4]:
URL = "https://y8.com"
response = requests.get(URL)

In [7]:
soup = BeautifulSoup(response.text, 'html.parser')

In [23]:
raw_data = []
cat_container = soup.find_all('div', class_='top-categories')[0]
for x in cat_container.find_all('a', href=True)[1:]:
    print(x['href'], x.text.split('\n')[0], x.text.split('\n')[1].split(' ')[0].replace(',',''))
    raw_data.append({'cat_url':x['href'], 
                     'cat_name':x.text.split('\n')[0], 
                     'cat_ngames': x.text.split('\n')[1].split(' ')[0].replace(',','')})
df_cat = pd.DataFrame(raw_data)
df_cat

https://www.y8.com/categories/girls For Girls 24277
https://www.y8.com/categories/shooting Shooting 9165
https://www.y8.com/categories/driving_racing Driving & Racing 6445
https://www.y8.com/categories/fighting Fighting 2088
https://www.y8.com/categories/strategy Strategy & RPG 3647
https://www.y8.com/categories/sports Sports 3376
https://www.y8.com/categories/management Management & Sim 4294
https://www.y8.com/categories/thinking Thinking 12898
https://www.y8.com/categories/action_adventure Action & Adventure 14849
https://www.y8.com/categories/fun Fun & Crazy 5006
https://www.y8.com/categories/skill Skill 11611
https://www.y8.com/categories/arcade_classic Arcade & Classic 9642


,cat_url,cat_name,cat_ngames
0,https://www.y8.com/categories/girls,For Girls,24277
1,https://www.y8.com/categories/shooting,Shooting,9165
2,https://www.y8.com/categories/driving_racing,Driving & Racing,6445
3,https://www.y8.com/categories/fighting,Fighting,2088
4,https://www.y8.com/categories/strategy,Strategy & RPG,3647
5,https://www.y8.com/categories/sports,Sports,3376
6,https://www.y8.com/categories/management,Management & Sim,4294
7,https://www.y8.com/categories/thinking,Thinking,12898
8,https://www.y8.com/categories/action_adventure,Action & Adventure,14849
9,https://www.y8.com/categories/fun,Fun & Crazy,5006


### y8.com -- get tags

In [24]:
URL = "https://y8.com/tags"
response = requests.get(URL)

In [25]:
soup = BeautifulSoup(response.text, 'html.parser')

In [32]:
raw_data = []
tag_container = soup.find_all('ul', class_='tag')[0]
for x in tag_container.find_all('li'):
    print(x)
    print(x.find_all('a')[0]['href'], x.find_all('a')[0]['title'])
    raw_data.append({'tag_url':x.find_all('a')[0]['href'], 
                     'tag_name':x.find_all('a')[0]['title']})
df_tag = pd.DataFrame(raw_data)
df_tag

<li><a class="tag" href="https://www.y8.com/tags/io_games" title=".io Games"><i class="tag-36x28 tag-36x28-games tag-36x28-1150"></i>.io Games<span style="font-size:13px;">119</span></a></li>
https://www.y8.com/tags/io_games .io Games
<li><a class="tag" href="https://www.y8.com/tags/1_player" title="1 player"><i class="tag-36x28 tag-36x28-games tag-36x28-10"></i>1 player<span style="font-size:13px;">77,442</span></a></li>
https://www.y8.com/tags/1_player 1 player
<li><a class="tag" href="https://www.y8.com/tags/2_players" title="2 player"><i class="tag-36x28 tag-36x28-games tag-36x28-5"></i>2 player<span style="font-size:13px;">1,297</span></a></li>
https://www.y8.com/tags/2_players 2 player
<li><a class="tag" href="https://www.y8.com/tags/3_players" title="3+ Player"><i class="tag-36x28 tag-36x28-games tag-36x28-331"></i>3+ Player<span style="font-size:13px;">99</span></a></li>
https://www.y8.com/tags/3_players 3+ Player
<li><a class="tag" href="https://www.y8.com/tags/3d" title="3D">

,tag_url,tag_name
0,https://www.y8.com/tags/io_games,.io Games
1,https://www.y8.com/tags/1_player,1 player
2,https://www.y8.com/tags/2_players,2 player
3,https://www.y8.com/tags/3_players,3+ Player
4,https://www.y8.com/tags/3d,3D
...,...,...
383,https://www.y8.com/tags/y8_save,Y8 Save
384,https://www.y8.com/tags/y8_screenshot,Y8 Screenshot
385,https://www.y8.com/tags/yahoo,Yahoo
386,https://www.y8.com/tags/zodiac,Zodiac


### Crawl by category

In [57]:
raw_data = []


for i, row in df_cat.iterrows():
    page_no = 1
    crawled_games = set()
    while True:
        print(row['cat_name'], row['cat_url'], "PAGE=", page_no)
        params = {'page':page_no}
        #requext next page
        response = requests.get(row['cat_url'], params=params)
        print(response.url)
        #parse results
        soup = BeautifulSoup(response.text, 'html.parser')
        games_boxes = soup.find_all('div', class_='item thumb videobox')
        if len(games_boxes)==0:
            break
        new_games = set()
        for x in games_boxes:
#             print(x)
            info = x.find('div', class_='infos')
            gametitle = info.find('p', class_='title ltr').text
            tech = info.find('div', class_='technology').find('p').text.split('\n')[1].lstrip()
            rating = info.find('p', class_='rating').find('span').text.split('\n')[1].lstrip()
            playscount =  info.find('p', class_='plays-count').text.split('\n')[1].lstrip()
            gameurl = x.find('a', href=True)['href']
#             print(gametitle, tech, rating, playscount, gameurl)
            raw_data.append({'game':gametitle, 
                            'url':gameurl, 
                            'tech':tech, 
                            'category':row['cat_name'], 
                            'rating':rating, 
                            'plays_count':playscount, 
                            'tag':'NO_TAG'})
            new_games.add(gametitle)
        if new_games - crawled_games == set(): #new results
            #stop increasing page
            break
            
        crawled_games = crawled_games | new_games #union
        page_no+=1
        
df_res_cat = pd.DataFrame(raw_data)
df_res_cat.to_csv('y8_game_info.csv')

For Girls https://www.y8.com/categories/girls PAGE= 1
https://www.y8.com/categories/girls?page=1
For Girls https://www.y8.com/categories/girls PAGE= 2
https://www.y8.com/categories/girls?page=2
For Girls https://www.y8.com/categories/girls PAGE= 3
https://www.y8.com/categories/girls?page=3
For Girls https://www.y8.com/categories/girls PAGE= 4
https://www.y8.com/categories/girls?page=4
For Girls https://www.y8.com/categories/girls PAGE= 5
https://www.y8.com/categories/girls?page=5
For Girls https://www.y8.com/categories/girls PAGE= 6
https://www.y8.com/categories/girls?page=6
For Girls https://www.y8.com/categories/girls PAGE= 7
https://www.y8.com/categories/girls?page=7
For Girls https://www.y8.com/categories/girls PAGE= 8
https://www.y8.com/categories/girls?page=8
For Girls https://www.y8.com/categories/girls PAGE= 9
https://www.y8.com/categories/girls?page=9
For Girls https://www.y8.com/categories/girls PAGE= 10
https://www.y8.com/categories/girls?page=10
For Girls https://www.y8.com

https://www.y8.com/categories/girls?page=84
For Girls https://www.y8.com/categories/girls PAGE= 85
https://www.y8.com/categories/girls?page=85
For Girls https://www.y8.com/categories/girls PAGE= 86
https://www.y8.com/categories/girls?page=86
For Girls https://www.y8.com/categories/girls PAGE= 87
https://www.y8.com/categories/girls?page=87
For Girls https://www.y8.com/categories/girls PAGE= 88
https://www.y8.com/categories/girls?page=88
For Girls https://www.y8.com/categories/girls PAGE= 89
https://www.y8.com/categories/girls?page=89
For Girls https://www.y8.com/categories/girls PAGE= 90
https://www.y8.com/categories/girls?page=90
For Girls https://www.y8.com/categories/girls PAGE= 91
https://www.y8.com/categories/girls?page=91
For Girls https://www.y8.com/categories/girls PAGE= 92
https://www.y8.com/categories/girls?page=92
For Girls https://www.y8.com/categories/girls PAGE= 93
https://www.y8.com/categories/girls?page=93
For Girls https://www.y8.com/categories/girls PAGE= 94
https://ww

https://www.y8.com/categories/shooting?page=14
Shooting https://www.y8.com/categories/shooting PAGE= 15
https://www.y8.com/categories/shooting?page=15
Shooting https://www.y8.com/categories/shooting PAGE= 16
https://www.y8.com/categories/shooting?page=16
Shooting https://www.y8.com/categories/shooting PAGE= 17
https://www.y8.com/categories/shooting?page=17
Shooting https://www.y8.com/categories/shooting PAGE= 18
https://www.y8.com/categories/shooting?page=18
Shooting https://www.y8.com/categories/shooting PAGE= 19
https://www.y8.com/categories/shooting?page=19
Shooting https://www.y8.com/categories/shooting PAGE= 20
https://www.y8.com/categories/shooting?page=20
Shooting https://www.y8.com/categories/shooting PAGE= 21
https://www.y8.com/categories/shooting?page=21
Shooting https://www.y8.com/categories/shooting PAGE= 22
https://www.y8.com/categories/shooting?page=22
Shooting https://www.y8.com/categories/shooting PAGE= 23
https://www.y8.com/categories/shooting?page=23
Shooting https://

https://www.y8.com/categories/shooting?page=93
Shooting https://www.y8.com/categories/shooting PAGE= 94
https://www.y8.com/categories/shooting?page=94
Shooting https://www.y8.com/categories/shooting PAGE= 95
https://www.y8.com/categories/shooting?page=95
Shooting https://www.y8.com/categories/shooting PAGE= 96
https://www.y8.com/categories/shooting?page=96
Shooting https://www.y8.com/categories/shooting PAGE= 97
https://www.y8.com/categories/shooting?page=97
Shooting https://www.y8.com/categories/shooting PAGE= 98
https://www.y8.com/categories/shooting?page=98
Shooting https://www.y8.com/categories/shooting PAGE= 99
https://www.y8.com/categories/shooting?page=99
Shooting https://www.y8.com/categories/shooting PAGE= 100
https://www.y8.com/categories/shooting?page=100
Shooting https://www.y8.com/categories/shooting PAGE= 101
https://www.y8.com/categories/shooting?page=101
Shooting https://www.y8.com/categories/shooting PAGE= 102
https://www.y8.com/categories/shooting?page=102
Shooting ht

https://www.y8.com/categories/driving_racing?page=22
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 23
https://www.y8.com/categories/driving_racing?page=23
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 24
https://www.y8.com/categories/driving_racing?page=24
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 25
https://www.y8.com/categories/driving_racing?page=25
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 26
https://www.y8.com/categories/driving_racing?page=26
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 27
https://www.y8.com/categories/driving_racing?page=27
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 28
https://www.y8.com/categories/driving_racing?page=28
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 29
https://www.y8.com/categories/driving_racing?page=29
Driving & Racing https://www.y8.com/categories/driving_racing PAGE= 30
https://

...

https://www.y8.com/categories/fun?page=59
Fun & Crazy https://www.y8.com/categories/fun PAGE= 60
https://www.y8.com/categories/fun?page=60
Fun & Crazy https://www.y8.com/categories/fun PAGE= 61
https://www.y8.com/categories/fun?page=61
Fun & Crazy https://www.y8.com/categories/fun PAGE= 62
https://www.y8.com/categories/fun?page=62
Fun & Crazy https://www.y8.com/categories/fun PAGE= 63
https://www.y8.com/categories/fun?page=63
Fun & Crazy https://www.y8.com/categories/fun PAGE= 64
https://www.y8.com/categories/fun?page=64
Fun & Crazy https://www.y8.com/categories/fun PAGE= 65
https://www.y8.com/categories/fun?page=65
Fun & Crazy https://www.y8.com/categories/fun PAGE= 66
https://www.y8.com/categories/fun?page=66
Fun & Crazy https://www.y8.com/categories/fun PAGE= 67
https://www.y8.com/categories/fun?page=67
Fun & Crazy https://www.y8.com/categories/fun PAGE= 68
https://www.y8.com/categories/fun?page=68
Fun & Crazy https://www.y8.com/categories/fun PAGE= 69
https://www.y8.com/categories/

https://www.y8.com/categories/skill?page=66
Skill https://www.y8.com/categories/skill PAGE= 67
https://www.y8.com/categories/skill?page=67
Skill https://www.y8.com/categories/skill PAGE= 68
https://www.y8.com/categories/skill?page=68
Skill https://www.y8.com/categories/skill PAGE= 69
https://www.y8.com/categories/skill?page=69
Skill https://www.y8.com/categories/skill PAGE= 70
https://www.y8.com/categories/skill?page=70
Skill https://www.y8.com/categories/skill PAGE= 71
https://www.y8.com/categories/skill?page=71
Skill https://www.y8.com/categories/skill PAGE= 72
https://www.y8.com/categories/skill?page=72
Skill https://www.y8.com/categories/skill PAGE= 73
https://www.y8.com/categories/skill?page=73
Skill https://www.y8.com/categories/skill PAGE= 74
https://www.y8.com/categories/skill?page=74
Skill https://www.y8.com/categories/skill PAGE= 75
https://www.y8.com/categories/skill?page=75
Skill https://www.y8.com/categories/skill PAGE= 76
https://www.y8.com/categories/skill?page=76
Skill 

https://www.y8.com/categories/arcade_classic?page=1
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 2
https://www.y8.com/categories/arcade_classic?page=2
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 3
https://www.y8.com/categories/arcade_classic?page=3
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 4
https://www.y8.com/categories/arcade_classic?page=4
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 5
https://www.y8.com/categories/arcade_classic?page=5
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 6
https://www.y8.com/categories/arcade_classic?page=6
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 7
https://www.y8.com/categories/arcade_classic?page=7
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 8
https://www.y8.com/categories/arcade_classic?page=8
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 9
https://www.y8.com/categ

https://www.y8.com/categories/arcade_classic?page=68
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 69
https://www.y8.com/categories/arcade_classic?page=69
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 70
https://www.y8.com/categories/arcade_classic?page=70
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 71
https://www.y8.com/categories/arcade_classic?page=71
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 72
https://www.y8.com/categories/arcade_classic?page=72
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 73
https://www.y8.com/categories/arcade_classic?page=73
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 74
https://www.y8.com/categories/arcade_classic?page=74
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 75
https://www.y8.com/categories/arcade_classic?page=75
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 76
https://

https://www.y8.com/categories/arcade_classic?page=134
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 135
https://www.y8.com/categories/arcade_classic?page=135
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 136
https://www.y8.com/categories/arcade_classic?page=136
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 137
https://www.y8.com/categories/arcade_classic?page=137
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 138
https://www.y8.com/categories/arcade_classic?page=138
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 139
https://www.y8.com/categories/arcade_classic?page=139
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 140
https://www.y8.com/categories/arcade_classic?page=140
Arcade & Classic https://www.y8.com/categories/arcade_classic PAGE= 141
https://www.y8.com/categories/arcade_classic?page=141
Arcade & Classic https://www.y8.com/categories/arcade_classic PA

### Crawl by tag

In [ ]:
raw_data = []


for i, row in df_tag.iterrows():
    page_no = 1
    crawled_games = set()
    while True:
        print(row['tag_name'], row['tag_url'], "PAGE=", page_no)
        params = {'page':page_no}
        #requext next page
        response = requests.get(row['tag_url'], params=params)
        print(response.url)
        #parse results
        soup = BeautifulSoup(response.text, 'html.parser')
        games_boxes = soup.find_all('div', class_='item thumb videobox')
        if len(games_boxes)==0:
            break
        new_games = set()
        for x in games_boxes:
#             print(x)
            info = x.find('div', class_='infos')
            gametitle = info.find('p', class_='title ltr').text
            tech = info.find('div', class_='technology').find('p').text.split('\n')[1].lstrip()
            rating = info.find('p', class_='rating').find('span').text.split('\n')[1].lstrip()
            playscount =  info.find('p', class_='plays-count').text.split('\n')[1].lstrip()
            gameurl = x.find('a', href=True)['href']
#             print(gametitle, tech, rating, playscount, gameurl)
            raw_data.append({'game':gametitle, 
                            'url':gameurl, 
                            'tech':tech, 
                            'category':"NO_CAT", 
                            'rating':rating, 
                            'plays_count':playscount, 
                            'tag':row['tag_name']})
            new_games.add(gametitle)
        if new_games - crawled_games == set(): #new results
            #stop increasing page
            break
            
        crawled_games = crawled_games | new_games #union
        page_no+=1
        
df_res_cat = pd.DataFrame(raw_data)
df_res_cat.to_csv('y8_game_info2.csv')

.io Games https://www.y8.com/tags/io_games PAGE= 1
https://www.y8.com/tags/io_games?page=1
.io Games https://www.y8.com/tags/io_games PAGE= 2
https://www.y8.com/tags/io_games?page=2
.io Games https://www.y8.com/tags/io_games PAGE= 3
https://www.y8.com/tags/io_games?page=3
1 player https://www.y8.com/tags/1_player PAGE= 1
https://www.y8.com/tags/1_player?page=1
1 player https://www.y8.com/tags/1_player PAGE= 2
https://www.y8.com/tags/1_player?page=2
1 player https://www.y8.com/tags/1_player PAGE= 3
https://www.y8.com/tags/1_player?page=3
1 player https://www.y8.com/tags/1_player PAGE= 4
https://www.y8.com/tags/1_player?page=4
1 player https://www.y8.com/tags/1_player PAGE= 5
https://www.y8.com/tags/1_player?page=5
1 player https://www.y8.com/tags/1_player PAGE= 6
https://www.y8.com/tags/1_player?page=6
1 player https://www.y8.com/tags/1_player PAGE= 7
https://www.y8.com/tags/1_player?page=7
1 player https://www.y8.com/tags/1_player PAGE= 8
https://www.y8.com/tags/1_player?page=8
1 playe

https://www.y8.com/tags/1_player?page=87
1 player https://www.y8.com/tags/1_player PAGE= 88
https://www.y8.com/tags/1_player?page=88
1 player https://www.y8.com/tags/1_player PAGE= 89
https://www.y8.com/tags/1_player?page=89
1 player https://www.y8.com/tags/1_player PAGE= 90
https://www.y8.com/tags/1_player?page=90
1 player https://www.y8.com/tags/1_player PAGE= 91
https://www.y8.com/tags/1_player?page=91
1 player https://www.y8.com/tags/1_player PAGE= 92
https://www.y8.com/tags/1_player?page=92
1 player https://www.y8.com/tags/1_player PAGE= 93
https://www.y8.com/tags/1_player?page=93
1 player https://www.y8.com/tags/1_player PAGE= 94
https://www.y8.com/tags/1_player?page=94
1 player https://www.y8.com/tags/1_player PAGE= 95
https://www.y8.com/tags/1_player?page=95
1 player https://www.y8.com/tags/1_player PAGE= 96
https://www.y8.com/tags/1_player?page=96
1 player https://www.y8.com/tags/1_player PAGE= 97
https://www.y8.com/tags/1_player?page=97
1 player https://www.y8.com/tags/1_play

https://www.y8.com/tags/3_players?page=2
3+ Player https://www.y8.com/tags/3_players PAGE= 3
https://www.y8.com/tags/3_players?page=3
3D https://www.y8.com/tags/3d PAGE= 1
https://www.y8.com/tags/3d?page=1
3D https://www.y8.com/tags/3d PAGE= 2
https://www.y8.com/tags/3d?page=2
3D https://www.y8.com/tags/3d PAGE= 3
https://www.y8.com/tags/3d?page=3
3D https://www.y8.com/tags/3d PAGE= 4
https://www.y8.com/tags/3d?page=4
3D https://www.y8.com/tags/3d PAGE= 5
https://www.y8.com/tags/3d?page=5
3D https://www.y8.com/tags/3d PAGE= 6
https://www.y8.com/tags/3d?page=6
3D https://www.y8.com/tags/3d PAGE= 7
https://www.y8.com/tags/3d?page=7
3D https://www.y8.com/tags/3d PAGE= 8
https://www.y8.com/tags/3d?page=8
3D https://www.y8.com/tags/3d PAGE= 9
https://www.y8.com/tags/3d?page=9
3D https://www.y8.com/tags/3d PAGE= 10
https://www.y8.com/tags/3d?page=10
3D https://www.y8.com/tags/3d PAGE= 11
https://www.y8.com/tags/3d?page=11
3D https://www.y8.com/tags/3d PAGE= 12
https://www.y8.com/tags/3d?page

https://www.y8.com/tags/action?page=38
Action https://www.y8.com/tags/action PAGE= 39
https://www.y8.com/tags/action?page=39
Action https://www.y8.com/tags/action PAGE= 40
https://www.y8.com/tags/action?page=40
Action https://www.y8.com/tags/action PAGE= 41
https://www.y8.com/tags/action?page=41
Action https://www.y8.com/tags/action PAGE= 42
https://www.y8.com/tags/action?page=42
Action https://www.y8.com/tags/action PAGE= 43
https://www.y8.com/tags/action?page=43
Action https://www.y8.com/tags/action PAGE= 44
https://www.y8.com/tags/action?page=44
Action https://www.y8.com/tags/action PAGE= 45
https://www.y8.com/tags/action?page=45
Action https://www.y8.com/tags/action PAGE= 46
https://www.y8.com/tags/action?page=46
Action https://www.y8.com/tags/action PAGE= 47
https://www.y8.com/tags/action?page=47
Action https://www.y8.com/tags/action PAGE= 48
https://www.y8.com/tags/action?page=48
Action https://www.y8.com/tags/action PAGE= 49
https://www.y8.com/tags/action?page=49
Action https://

https://www.y8.com/tags/addition?page=5
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 1
https://www.y8.com/tags/adrenaline?page=1
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 2
https://www.y8.com/tags/adrenaline?page=2
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 3
https://www.y8.com/tags/adrenaline?page=3
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 4
https://www.y8.com/tags/adrenaline?page=4
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 5
https://www.y8.com/tags/adrenaline?page=5
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 6
https://www.y8.com/tags/adrenaline?page=6
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 7
https://www.y8.com/tags/adrenaline?page=7
Adrenaline https://www.y8.com/tags/adrenaline PAGE= 8
https://www.y8.com/tags/adrenaline?page=8
Adventure https://www.y8.com/tags/adventure PAGE= 1
https://www.y8.com/tags/adventure?page=1
Adventure https://www.y8.com/tags/adventure PAGE= 2
https://www.y8.com/tags/adventure?page=2
Advent

...

https://www.y8.com/tags/html5?page=48
HTML5 https://www.y8.com/tags/html5 PAGE= 49
https://www.y8.com/tags/html5?page=49
HTML5 https://www.y8.com/tags/html5 PAGE= 50
https://www.y8.com/tags/html5?page=50
HTML5 https://www.y8.com/tags/html5 PAGE= 51
https://www.y8.com/tags/html5?page=51
HTML5 https://www.y8.com/tags/html5 PAGE= 52
https://www.y8.com/tags/html5?page=52
HTML5 https://www.y8.com/tags/html5 PAGE= 53
https://www.y8.com/tags/html5?page=53
HTML5 https://www.y8.com/tags/html5 PAGE= 54
https://www.y8.com/tags/html5?page=54
HTML5 https://www.y8.com/tags/html5 PAGE= 55
https://www.y8.com/tags/html5?page=55
HTML5 https://www.y8.com/tags/html5 PAGE= 56
https://www.y8.com/tags/html5?page=56
HTML5 https://www.y8.com/tags/html5 PAGE= 57
https://www.y8.com/tags/html5?page=57
HTML5 https://www.y8.com/tags/html5 PAGE= 58
https://www.y8.com/tags/html5?page=58
HTML5 https://www.y8.com/tags/html5 PAGE= 59
https://www.y8.com/tags/html5?page=59
HTML5 https://www.y8.com/tags/html5 PAGE= 60
http

https://www.y8.com/tags/html5?page=146
HTML5 https://www.y8.com/tags/html5 PAGE= 147
https://www.y8.com/tags/html5?page=147
HTML5 https://www.y8.com/tags/html5 PAGE= 148
https://www.y8.com/tags/html5?page=148
HTML5 https://www.y8.com/tags/html5 PAGE= 149
https://www.y8.com/tags/html5?page=149
HTML5 https://www.y8.com/tags/html5 PAGE= 150
https://www.y8.com/tags/html5?page=150
Hunting https://www.y8.com/tags/hunting PAGE= 1
https://www.y8.com/tags/hunting?page=1
Hunting https://www.y8.com/tags/hunting PAGE= 2
https://www.y8.com/tags/hunting?page=2
Ice https://www.y8.com/tags/ice PAGE= 1
https://www.y8.com/tags/ice?page=1
Ice https://www.y8.com/tags/ice PAGE= 2
https://www.y8.com/tags/ice?page=2
Ice https://www.y8.com/tags/ice PAGE= 3
https://www.y8.com/tags/ice?page=3
Ice https://www.y8.com/tags/ice PAGE= 4
https://www.y8.com/tags/ice?page=4
Ice https://www.y8.com/tags/ice PAGE= 5
https://www.y8.com/tags/ice?page=5
Ice https://www.y8.com/tags/ice PAGE= 6
https://www.y8.com/tags/ice?page

https://www.y8.com/tags/ipad?page=65
iPad https://www.y8.com/tags/ipad PAGE= 66
https://www.y8.com/tags/ipad?page=66
iPad https://www.y8.com/tags/ipad PAGE= 67
https://www.y8.com/tags/ipad?page=67
iPad https://www.y8.com/tags/ipad PAGE= 68
https://www.y8.com/tags/ipad?page=68
iPad https://www.y8.com/tags/ipad PAGE= 69
https://www.y8.com/tags/ipad?page=69
iPad https://www.y8.com/tags/ipad PAGE= 70
https://www.y8.com/tags/ipad?page=70
iPad https://www.y8.com/tags/ipad PAGE= 71
https://www.y8.com/tags/ipad?page=71
iPad https://www.y8.com/tags/ipad PAGE= 72
https://www.y8.com/tags/ipad?page=72
iPad https://www.y8.com/tags/ipad PAGE= 73
https://www.y8.com/tags/ipad?page=73
iPad https://www.y8.com/tags/ipad PAGE= 74
https://www.y8.com/tags/ipad?page=74
iPad https://www.y8.com/tags/ipad PAGE= 75
https://www.y8.com/tags/ipad?page=75
iPad https://www.y8.com/tags/ipad PAGE= 76
https://www.y8.com/tags/ipad?page=76
iPad https://www.y8.com/tags/ipad PAGE= 77
https://www.y8.com/tags/ipad?page=77
iPa

https://www.y8.com/tags/iphone?page=58
iPhone https://www.y8.com/tags/iphone PAGE= 59
https://www.y8.com/tags/iphone?page=59
iPhone https://www.y8.com/tags/iphone PAGE= 60
https://www.y8.com/tags/iphone?page=60
iPhone https://www.y8.com/tags/iphone PAGE= 61
https://www.y8.com/tags/iphone?page=61
iPhone https://www.y8.com/tags/iphone PAGE= 62
https://www.y8.com/tags/iphone?page=62
iPhone https://www.y8.com/tags/iphone PAGE= 63
https://www.y8.com/tags/iphone?page=63
iPhone https://www.y8.com/tags/iphone PAGE= 64
https://www.y8.com/tags/iphone?page=64
iPhone https://www.y8.com/tags/iphone PAGE= 65
https://www.y8.com/tags/iphone?page=65
iPhone https://www.y8.com/tags/iphone PAGE= 66
https://www.y8.com/tags/iphone?page=66
iPhone https://www.y8.com/tags/iphone PAGE= 67
https://www.y8.com/tags/iphone?page=67
iPhone https://www.y8.com/tags/iphone PAGE= 68
https://www.y8.com/tags/iphone?page=68
iPhone https://www.y8.com/tags/iphone PAGE= 69
https://www.y8.com/tags/iphone?page=69
iPhone https://

https://www.y8.com/tags/jumping?page=18
Jumping https://www.y8.com/tags/jumping PAGE= 19
https://www.y8.com/tags/jumping?page=19
Jumping https://www.y8.com/tags/jumping PAGE= 20
https://www.y8.com/tags/jumping?page=20
Jumping https://www.y8.com/tags/jumping PAGE= 21
https://www.y8.com/tags/jumping?page=21
Jumping https://www.y8.com/tags/jumping PAGE= 22
https://www.y8.com/tags/jumping?page=22
Jumping https://www.y8.com/tags/jumping PAGE= 23
https://www.y8.com/tags/jumping?page=23
Jumping https://www.y8.com/tags/jumping PAGE= 24
https://www.y8.com/tags/jumping?page=24
Jumping https://www.y8.com/tags/jumping PAGE= 25
https://www.y8.com/tags/jumping?page=25
Jumping https://www.y8.com/tags/jumping PAGE= 26
https://www.y8.com/tags/jumping?page=26
Jumping https://www.y8.com/tags/jumping PAGE= 27
https://www.y8.com/tags/jumping?page=27
Jumping https://www.y8.com/tags/jumping PAGE= 28
https://www.y8.com/tags/jumping?page=28
Jumping https://www.y8.com/tags/jumping PAGE= 29
https://www.y8.com/ta

In [60]:
df_res_cat

,game,url,tech,category,rating,plays_count,tag
0,Top Shooter io,https://www.y8.com/games/top_shooter_io,WebGL,NO_CAT,79%,"7,119 plays",.io Games
1,Hole io,https://www.y8.com/games/hole_io,WebGL,NO_CAT,86%,"948,380 plays",.io Games
2,Aquapark,https://www.y8.com/games/aquapark,HTML5,NO_CAT,72%,"372,239 plays",.io Games
3,Baby Shark io,https://www.y8.com/games/baby_shark_io,HTML5,NO_CAT,62%,"4,257 plays",.io Games
4,Inkwars io,https://www.y8.com/games/inkwars_io,WebGL,NO_CAT,75%,"5,058 plays",.io Games
...,...,...,...,...,...,...,...
388385,Fright Club,https://www.y8.com/games/fright_club,Flash,NO_CAT,81%,"2,248 plays",Zombie
388386,Organ Trail,https://www.y8.com/games/organ_trail,Flash,NO_CAT,71%,"6,086 plays",Zombie
388387,Undead Parade,https://www.y8.com/games/undead_parade,Flash,NO_CAT,81%,"2,564 plays",Zombie
388388,Ninja Kot,https://www.y8.com/games/ninja_kot,Flash,NO_CAT,81%,"3,578 plays",Zombie


In [62]:
df_res_cat['game'].nunique()

77475

In [64]:
pd.read_csv("y8_game_info.csv", index_col=0).drop_duplicates()

,game,url,tech,category,rating,plays_count,tag
0,Villains School OOTD,https://www.y8.com/games/villains_school_ootd,HTML5,For Girls,80%,86 plays,NO_TAG
1,My Cute Room Decor,https://www.y8.com/games/my_cute_room_decor,HTML5,For Girls,78%,"42,658 plays",NO_TAG
2,Popular Girl,https://www.y8.com/games/popular_girl,HTML5,For Girls,88%,"35,986 plays",NO_TAG
3,Smoothie Maker,https://www.y8.com/games/smoothie_maker_,WebGL,For Girls,66%,362 plays,NO_TAG
4,Children Laundry,https://www.y8.com/games/children_laundry,HTML5,For Girls,81%,"3,064,845 plays",NO_TAG
...,...,...,...,...,...,...,...
82328,Midnight Serenade,https://www.y8.com/games/midnight_serenade,Flash,Arcade & Classic,73%,"28,754 plays",NO_TAG
82329,Checkers Fun!,https://www.y8.com/games/checkers_fun_,Flash,Arcade & Classic,80%,"3,554,507 plays",NO_TAG
82330,Bomb of Braveboy,https://www.y8.com/games/bomb_of_braveboy,Flash,Arcade & Classic,85%,"2,042,355 plays",NO_TAG
82331,Harry the Hamster,https://www.y8.com/games/harry_the_hamster,Flash,Arcade & Classic,50%,"202,126 plays",NO_TAG
